In [ ]:
import sys
import os

# Add the project root directory to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

# Now you can import the IntentClassifier from src/infer_intent.py
from src.infer_intent import IntentClassifier
from tqdm import tqdm
import pandas as pd

In [ ]:
with open('../data/internal/search_examples.txt', 'r', encoding='utf-8') as f:
    yelp_texts = f.read().split('\n')

In [ ]:
yelp_texts[:10]

In [ ]:
!ls -ltrsh models/

In [ ]:
cls = IntentClassifier()

#### Yelp internal data evaluation

In [ ]:
yelp_hit_count = 0
yelp_queries_misclassified = []
for query in tqdm(yelp_texts):
    pred_result, _ = cls.find_intent(query)
    if pred_result == 'yelp_intent':
        yelp_hit_count += 1
    else:
        yelp_queries_misclassified.append({'query': query, 'pred_result': pred_result})
print(f"yelp hit rate (accuracy) = {yelp_hit_count/len(yelp_texts)}")

In [ ]:
## old
# yelp hit rate (accuracy) = 0.3256021409455843

## new (Mozilla/mobilebert-uncased-finetuned-LoRA-intent-classifier)
# yelp hit rate (accuracy) = 0.6257805530776093

In [ ]:
yelp_queries_misclassified_df = pd.DataFrame(yelp_queries_misclassified)
print(f"Number of yelp queries misclassified = {len(yelp_queries_misclassified_df)} out of {len(yelp_texts)} examples")
yelp_queries_misclassified_df

#### Weather internal data evaluation

In [ ]:
with open('../data/internal/weather_search_examples.txt', 'r', encoding='utf-8') as f:
    weather_texts = f.read().split('\n')

In [ ]:
weather_hit_count = 0
weather_queries_misclassified = []
for query in tqdm(weather_texts):
    pred_result, _ = cls.find_intent(query)
    if pred_result == 'weather_intent':
        weather_hit_count += 1
    else:
        weather_queries_misclassified.append({'query': query, 'pred_result': pred_result})
print(f"weather hit rate (accuracy) = {weather_hit_count/len(weather_texts)}")

In [ ]:
## old
# weather hit rate (accuracy) = 0.3902439024390244

## new (Mozilla/mobilebert-uncased-finetuned-LoRA-intent-classifier)
# weather hit rate (accuracy) = 0.6097560975609756

In [ ]:
weather_queries_misclassified_df = pd.DataFrame(weather_queries_misclassified)
print(f"Number of weather queries misclassified = {len(weather_queries_misclassified)} out of {len(weather_texts)} examples")
weather_queries_misclassified_df